<a href="https://colab.research.google.com/github/sheikmohdimran/Experiments_2020/blob/master/Vision/MoNuSAC_BRCA_Data_Curation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q gdown
!pip install -q libtiff
!apt-get install openslide-tools 
!apt-get install python3-openslide

In [0]:
import gdown

id = '<enter_id_here>'
url = 'https://drive.google.com/uc?id='+id
output = 'MoNuSAC_images_and_annotations.zip'
gdown.download(url, output, quiet=False)

!unzip -q MoNuSAC_images_and_annotations.zip

In [0]:
# Based on - https://github.com/ruchikaverma-iitg/MoNuSAC/blob/master/Binary_mask_generation.ipynb
import os
import openslide
from xml.dom import minidom
import numpy as np
import openslide
from openslide import open_slide  
from glob import glob
import cv2
import matplotlib.pyplot as plt
import scipy.io as sio
from PIL import Image
import scipy
import scipy.ndimage
from shapely.geometry import Polygon
from skimage import draw
import xml.etree.ElementTree as ET

In [83]:
# Read svs files from the desired path
!rm -rf /content/MoNuSAC_masks/

count = 0
data_path = '/content/MoNuSAC_images_and_annotations/' #Path to read data from
destination_path = '/content' # Path to save binary masks corresponding to xml files
os.chdir(destination_path)

try:
    os.mkdir(destination_path+'/MoNuSAC_masks')
except OSError:
    print ("Creation of the mask directory %s failed" % destination_path)
    
os.chdir(destination_path+'/MoNuSAC_masks')#Create folder named as MoNuSAC_masks
patients = [x[0] for x in os.walk(data_path)]#Total patients in the data_path
len(patients)

47

In [0]:
brca = ['TCGA-A2-A0CV-01Z-00-DX1','TCGA-A2-A0ES-01Z-00-DX1','TCGA-B6-A0WZ-01Z-00-DX1',
        'TCGA-BH-A18T-01Z-00-DX1','TCGA-D8-A1X5-01Z-00-DX2','TCGA-E2-A154-01Z-00-DX1',
        'TCGA-E9-A22B-01Z-00-DX1','TCGA-E9-A22G-01Z-00-DX1','TCGA-EW-A6SD-01Z-00-DX1',
        'TCGA-S3-AA11-01Z-00-DX1']

In [0]:
from pathlib import Path
wrkdir = Path('/content/MoNuSAC_masks/')
datadir = Path('/content/MoNuSAC_images_and_annotations/')

In [0]:
sub_images = list(datadir.glob('**/*.svs'))
for sub_image_loc in sub_images:
  # Filter only BRCA
  if sub_image_loc.stem.split("_")[0] in brca:
    sub_image_name = sub_image_loc.stem  
    sub_image = './'+sub_image_loc.stem+'/'+sub_image_name #Destination path
        
    image_name = str(sub_image_loc.with_suffix('.svs'))
    img = openslide.OpenSlide(image_name)
                              
    # If svs image needs to save in png
    plt.imsave(wrkdir/Path(sub_image_name).with_suffix('.png'), np.array(img.read_region((0,0),0,img.level_dimensions[0])))
    
    # Read xml file
    xml_file_name  = image_name[:-4]
    xml_file_name = xml_file_name+'.xml'
    tree = ET.parse(xml_file_name)
    root = tree.getroot()
    
    #Generate binary mask for each cell-type                         
    for k in range(len(root)):
        label = [x.attrib['Name'] for x in root[k][0]]
        label = label[0]
        
        for child in root[k]:
            for x in child:
                r = x.tag
                if r == 'Attribute':
                    count = count+1
                    #print(count)
                    label = x.attrib['Name']
                    binary_mask = np.transpose(np.zeros((img.read_region((0,0),0,img.level_dimensions[0]).size))) 
                                                          
                if (r == 'Region' and label == 'Lymphocyte'): #Only extract lymphocyte masks
                    regions = []
                    vertices = x[1]
                    coords = np.zeros((len(vertices), 2))
                    for i, vertex in enumerate(vertices):
                        coords[i][0] = vertex.attrib['X']
                        coords[i][1] = vertex.attrib['Y']        
                    regions.append(coords)
                    poly = Polygon(regions[0])  
                    
                    vertex_row_coords = regions[0][:,0]
                    vertex_col_coords = regions[0][:,1]
                    fill_row_coords, fill_col_coords = draw.polygon(vertex_col_coords, vertex_row_coords, binary_mask.shape)
                    binary_mask[fill_row_coords, fill_col_coords] = 1
                    mask_path = sub_image_name+'_'+label+'_mask.png'
                    plt.imsave(mask_path, binary_mask)


In [0]:
#from libtiff import TIFF
#import matplotlib.image as plt

#tif = TIFF.open('/content/MoNuSAC_masks/CGA-55-1594-01Z-00-DX1_003/Lymphocyte/2_mask.tif') 
#image = tif.read_image()
#plt.imsave('name.png', image)

In [0]:
#np.unique(image)

array([  0., 255.])

In [0]:
#!wget https://raw.githubusercontent.com/SBU-BMI/quip_classification/master/training_patch_extraction/list_wsi_on_gcloud.txt

In [0]:
#!cat list_wsi_on_gcloud.txt |grep brca > brca.txt

In [0]:
!rm -rf /content/BRCA
!mkdir -p /content/BRCA/masks
!mkdir -p /content/BRCA/images

!mv /content/MoNuSAC_masks/*mask*png /content/BRCA/masks/
!ls -1 /content/BRCA/masks/|awk -F "_" '{print "mv /content/MoNuSAC_masks/"$1"_"$2".png /content/BRCA/images/"}' > temp.sh
!sh temp.sh
!ls -1 /content/BRCA/masks/|wc -l

In [99]:
!tar -cvzf BRCA.tgz /content/BRCA/
!mv BRCA.tgz /content

tar: Removing leading `/' from member names
/content/BRCA/
/content/BRCA/masks/
/content/BRCA/masks/TCGA-E9-A22B-01Z-00-DX1_4_Lymphocyte_mask.png
/content/BRCA/masks/TCGA-D8-A1X5-01Z-00-DX2_1_Lymphocyte_mask.png
/content/BRCA/masks/TCGA-B6-A0WZ-01Z-00-DX1_3_Lymphocyte_mask.png
/content/BRCA/masks/TCGA-D8-A1X5-01Z-00-DX2_2_Lymphocyte_mask.png
/content/BRCA/masks/TCGA-BH-A18T-01Z-00-DX1_4_Lymphocyte_mask.png
/content/BRCA/masks/TCGA-E9-A22B-01Z-00-DX1_8_Lymphocyte_mask.png
/content/BRCA/masks/TCGA-EW-A6SD-01Z-00-DX1_3_Lymphocyte_mask.png
/content/BRCA/masks/TCGA-E9-A22G-01Z-00-DX1_3_Lymphocyte_mask.png
/content/BRCA/masks/TCGA-BH-A18T-01Z-00-DX1_3_Lymphocyte_mask.png
/content/BRCA/masks/TCGA-S3-AA11-01Z-00-DX1_6_Lymphocyte_mask.png
/content/BRCA/masks/TCGA-BH-A18T-01Z-00-DX1_6_Lymphocyte_mask.png
/content/BRCA/masks/TCGA-E9-A22B-01Z-00-DX1_1_Lymphocyte_mask.png
/content/BRCA/masks/TCGA-S3-AA11-01Z-00-DX1_2_Lymphocyte_mask.png
/content/BRCA/masks/TCGA-A2-A0ES-01Z-00-DX1_5_Lymphocyte_mask.

In [98]:
!pwd

/content/MoNuSAC_masks
